## Marker Selection

In [1]:
import pathlib
from concurrent.futures import ProcessPoolExecutor, as_completed
from itertools import combinations
import numpy as np
import anndata
import matplotlib.pyplot as plt
import pandas as pd
import scanpy as sc
import seaborn as sns
import xarray as xr
import subprocess
from sklearn.metrics import roc_auc_score
from cemba_data.tools.hdf5.anndata import rank_features_groups
import warnings
warnings.filterwarnings('ignore')

## Parameter

In [2]:
mcds_pattern = '/home/hanliu/project/mouse_rostral_brain/study/Level1-CellClass/ALL_manual/Adata/GeneWithSlop2kb.gene_da_rate.*.mcds'
min_cluster_cell_number = 10
adj_p_cutoff = 1e-3
top_n = 30000
max_test_cell_population = 1000
chunk_size = 100
delta_rate_cutoff = 0.5
auroc_cutoff = 0.8

In [3]:
# Parameters
cluster_col = "SubType"
use_clusters = ["MGE-Sst Rxra", "Unc5c Unc5c", "MGE-Sst Ptpre", "CGE-Lamp5 Dock5", "CGE-Vip Ntng1", "CGE-Vip Ptprm", "CGE-Vip Robo1", "MGE-Pvalb Gfra2", "PAL-Inh Meis2", "MGE-Sst Unc5b", "MGE-Pvalb Thsd7a", "CGE-Vip Grm8", "MGE-Sst Dock4", "CGE-Lamp5 Grk5", "OLF Xkr6", "MSN-D2 Slc24a2", "CGE-Lamp5 Sorcs1", "MGE-Pvalb Entpd3", "MGE-Pvalb Ptprk", "CGE-Vip Ccser1", "CGE-Lamp5 Grid1", "MGE-Pvalb Sema5a", "MGE-Sst Kcnip4", "MGE-Sst Frmd6", "MGE-Pvalb Cnih3", "MGE-Sst Ubtd1", "MGE-Sst Rerg", "CGE-Vip Fstl4", "CGE-Vip Galnt17", "MGE-Sst Etv1", "CGE-Vip Clstn2", "CGE-Lamp5 Nrxn3", "MGE-Sst Chodl", "OLF Trpc4", "Foxp2 Homer2", "MGE-Pvalb Cacna1i", "MSN-D1 Khdrbs3", "MSN-D1 Plxnc1", "OLF Mapk10", "MSN-D1 Hrh1", "Foxp2 Trpc7", "OLF Pag1", "MSN-D2 Col14a1", "MGE-Sst Bmper", "OLF Gabbr2", "OLF Kcnd3", "PAL-Inh Deptor", "PAL-Inh Meis1", "LSX-Inh Cacna1i", "D1L-Fstl4 Sipa1l2", "PAL-Inh Rarb", "MSN-D2 Nrp2", "D1L-Fstl4 Trps1", "Foxp2 Dchs2", "PAL-Inh Chat", "D1L-PAL Flrt2", "PAL-Inh Igdcc3", "PAL-Inh Tmem178", "MSN-D1 Ntn1", "Foxp2 Inpp4b", "MSN-D2 Casz1", "Chd7 Kcnc2", "PAL-Inh Tcf7l2", "D1L-Fstl4 Grm3", "D1L-Fstl4 Cadm1", "Chd7 Trpc7", "PAL-Inh Ptprd", "D1L-Fstl4 Crim1", "Chd7 Megf11", "D1L-PAL Plcxd3", "PAL-Inh Onecut2", "LSX-Inh Foxp2", "LSX-Inh Enox1", "LSX-Inh Dock10", "LSX-Inh Nxph1", "LSX-Inh Zeb2", "LSX-Inh Lats2"]
mcds_pattern = "/home/hanliu/project/mouse_rostral_brain/study/Level1-CellClass/ALL_manual/Adata/GeneWithSlop2kb.gene_da_rate.*.mcds"
min_cluster_cell_number = 10
adj_p_cutoff = 0.001
top_n = 30000
cpu = 20
max_test_cell_population = 2000
chunk_size = 20
delta_rate_cutoff = 0.3
auroc_cutoff = 0.8


### Stable Parameter

In [4]:
random_seed = 0

## Cell Tidy Data

In [5]:
tidy_data_path = '/home/hanliu/project/mouse_rostral_brain/study/ClusteringSummary/Summary/TotalClusteringResults.msg'

In [6]:
cell_tidy_data = pd.read_msgpack(tidy_data_path)

if use_clusters is not None:
    cell_tidy_data = cell_tidy_data[cell_tidy_data[cluster_col].isin(
        use_clusters)]
cell_tidy_data.shape[0]

28077

In [7]:
records = []
for cluster, sub_df in cell_tidy_data.groupby(cluster_col):
    if sub_df.shape[0] < max_test_cell_population:
        records.append(sub_df)
    else:
        records.append(
            sub_df.sample(max_test_cell_population, random_state=random_seed))
cell_tidy_data = pd.concat(records)
cell_tidy_data[cluster_col].value_counts()

MSN-D2 Slc24a2    2000
PAL-Inh Meis2     1969
MSN-D1 Hrh1       1349
OLF Trpc4         1017
MSN-D1 Plxnc1      926
                  ... 
Chd7 Kcnc2          79
CGE-Vip Ntng1       65
OLF Mapk10          60
OLF Pag1            60
OLF Xkr6            46
Name: SubType, Length: 77, dtype: int64

In [8]:
cluster_series = cell_tidy_data[cluster_col].astype(str)
cluster_counts = cluster_series.value_counts()


def check_cluster(cluster, count):
    if count < min_cluster_cell_number:
        return False
    return True


unique_clusters = [
    cluster for cluster, count in cluster_counts.items()
    if check_cluster(cluster, count)
]
cluster_pairs = list(combinations(unique_clusters, 2))

print(len(unique_clusters), 'pass filter.')
print(len(cluster_pairs), 'pairwise comparison to test.')

77 pass filter.
2926 pairwise comparison to test.


## Gene meta

In [9]:
gene_meta = pd.read_csv(
    '/home/hanliu/project/mouse_rostral_brain/study/ClusterMethylMarker/gencode.vM22.annotation.gene.flat.filtered_white_genes.tsv.gz',
    index_col='gene_id',
    sep='\t')
gene_meta.index.name = 'gene'
gene_name_to_id = {v: k for k, v in gene_meta['gene_name'].iteritems()}
gene_idbase_to_id = {i.split('.')[0]: i for i in gene_meta.index}

## Adata

In [10]:
gene_mcds = xr.open_mfdataset(mcds_pattern)
use_gene = gene_mcds.get_index('gene') & gene_meta.index
gene_meta = gene_meta.reindex(use_gene)

In [11]:
gene_mcds

<xarray.Dataset>
Dimensions:           (cell: 104340, gene: 55487, mc_type: 2)
Coordinates:
    strand_type       <U4 'both'
    geneslop2k_end    (gene) int64 dask.array<chunksize=(55487,), meta=np.ndarray>
    geneslop2k_chrom  (gene) object dask.array<chunksize=(55487,), meta=np.ndarray>
    geneslop2k_start  (gene) int64 dask.array<chunksize=(55487,), meta=np.ndarray>
  * gene              (gene) object 'ENSMUSG00000102693.1' ... 'ENSMUSG00000064372.1'
  * mc_type           (mc_type) object 'CGN' 'CHN'
  * cell              (cell) object '1A_M_0' '1A_M_1' ... '8J_M_1292'
Data variables:
    gene_da           (cell, gene, mc_type) float64 dask.array<chunksize=(10000, 55487, 2), meta=np.ndarray>

In [12]:
gene_mcds = gene_mcds['gene_da'].sel(mc_type='CHN',
                                     cell=cell_tidy_data.index,
                                     gene=gene_meta.index)
gene_mcds

<xarray.DataArray 'gene_da' (cell: 28008, gene: 50231)>
dask.array<getitem, shape=(28008, 50231), dtype=float64, chunksize=(738, 50231)>
Coordinates:
    strand_type       <U4 'both'
    geneslop2k_end    (gene) int64 dask.array<chunksize=(50231,), meta=np.ndarray>
    geneslop2k_chrom  (gene) object dask.array<chunksize=(50231,), meta=np.ndarray>
    geneslop2k_start  (gene) int64 dask.array<chunksize=(50231,), meta=np.ndarray>
  * gene              (gene) object 'ENSMUSG00000102693.1' ... 'ENSMUSG00000064372.1'
    mc_type           <U3 'CHN'
  * cell              (cell) object '10E_M_150' '10E_M_322' ... '9J_M_572'

In [13]:
cell_tidy_data.to_msgpack('TEMP.msg')

In [14]:
gene_mcds.to_netcdf('TEMP.nc')

## Pairwise test

In [10]:
def get_sig_features(rank_gene_dict):
    pvals_adj = pd.DataFrame(rank_gene_dict['pvals_adj'])
    names = pd.DataFrame(rank_gene_dict['names'])
    return pvals_adj, names


def get_delta(cluster, gene):
    row = cluster_mean_gene_df.loc[gene].copy()
    cluster_value = row.pop(cluster)
    other_mean = row[0] # only two cluster
    delta = cluster_value - other_mean
    return delta


def calculate_single_pair(data_path, pair):
    cluster_a, cluster_b = pair
    output_dir = pathlib.Path(f'TEMP/{cluster_a}')
    output_dir.mkdir(exist_ok=True)
    output_path = output_dir / f'{cluster_b}.msg'
    if output_path.exists():
        return
    
    this_tidy_data = pd.read_msgpack('TEMP.msg')
    this_tidy_data = this_tidy_data[this_tidy_data[cluster_col].isin(pair)]
    
    mcds = xr.open_dataarray(data_path).sel(cell=this_tidy_data.index).load()
    adata = anndata.AnnData(X=mcds.values,
                    obs=pd.DataFrame([], mcds.get_index('cell')),
                    var=pd.DataFrame([], mcds.get_index('gene')))
    adata.obs['cluster'] = this_tidy_data[cluster_col].astype('category')
    
    # reverse_adata, centered by 1 because after normalization all prior is center to 1
    adata.X = (adata.X - 1) * -1 + 1
    
    # calculate cluster delta
    records = {}
    for cluster, sub_df in adata.obs.groupby('cluster'):
        sub_adata = adata[sub_df.index, :]
        gene_mean = sub_adata.X.mean(axis=0)
        records[cluster] = pd.Series(gene_mean, index=sub_adata.var_names)
    cluster_mean_gene_df = pd.DataFrame(records)

    # judge gene by delta
    cluster_delta = (cluster_mean_gene_df[cluster_a] -
                     cluster_mean_gene_df[cluster_b]).abs()
    delta_judge = cluster_delta > delta_rate_cutoff
    use_adata = adata[:, delta_judge]
    
    sc.tl.rank_genes_groups(use_adata,
                            groupby='cluster',
                            n_genes=top_n,
                            method='wilcoxon')
    pvals_adj, names = get_sig_features(
        use_adata.uns['rank_genes_groups'])

    results = []
    for col in use_adata.obs['cluster'].unique():
        if col not in pair:
            continue
        df = pd.DataFrame({
            'pvals_adj': pvals_adj[col].tolist(),
            'gene_id': names[col].tolist()
        })
        df['cluster_from'] = col
        df['cluster_to'] = cluster_a if col == cluster_b else cluster_b
        results.append(df)
    
    # get total results filter by adj_p
    total_results = pd.concat(results)
    total_results['gene_name'] = total_results['gene_id'].map(
        gene_meta['gene_name'])
    total_results['-lgp'] = -np.log10(total_results['pvals_adj'])
    total_results['-lgp'] = total_results['-lgp'].replace(np.inf, 1000)
    total_results = total_results[
        total_results['pvals_adj'] < adj_p_cutoff].copy()

    # judge by auroc
    total_results['AUROC'] = total_results[[
        'gene_id', 'cluster_from'
    ]].apply(lambda i: get_auroc(i['gene_id'], i['cluster_from'],
                                 use_adata),
             axis=1)
    total_results = total_results[total_results['AUROC'] > auroc_cutoff]
    total_results.to_msgpack(output_path)
    return


def get_auroc(gene_id, cluster, adata):
    yscore = adata.obs_vector(gene_id)
    ylabel = adata.obs['cluster'] == cluster
    score = roc_auc_score(ylabel, yscore)
    score = abs(score - 0.5) + 0.5
    return score

In [11]:
data_path = 'TEMP.nc'

## Run pairwise marker

In [13]:
pair_marker_counts = {}
n = 0
for chunk_start in range(0, len(cluster_pairs), chunk_size):
    pairs = cluster_pairs[chunk_start : chunk_start + chunk_size]
    
    with ProcessPoolExecutor(cpu) as executor:
        temp_dir = 'TEMP'
        pathlib.Path(temp_dir).mkdir(exist_ok=True)
        futures = []
        for pair in pairs:
            future = executor.submit(calculate_single_pair, data_path, pair)
            futures.append(future)
    
        for future in as_completed(futures):
            n += 1
            if n % 100 == 0:
                print(n)
            future.result()

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900


## Aggregate DEG

In [14]:
temp_dir = pathlib.Path('TEMP')
deg_list = list(temp_dir.glob('**/*msg'))

df_list = []
for path in deg_list:
    df_list.append(pd.read_msgpack(path))

total_markers = pd.concat(df_list)

## Add rate

In [15]:
gene_mean_mch = pd.read_msgpack(
    '/home/hanliu/project/mouse_rostral_brain/study/AssignGeneToTree/SubType.gene_mean_mch.msg')

total_markers['cluster_from_rate'] = total_markers.apply(
    lambda i: gene_mean_mch.at[i['cluster_from'], i['gene_id']], axis=1)
total_markers['cluster_to_rate'] = total_markers.apply(
    lambda i: gene_mean_mch.at[i['cluster_to'], i['gene_id']], axis=1)

## Filter

In [16]:
total_markers['log2fc'] = np.log2(total_markers['cluster_from_rate'] / total_markers['cluster_to_rate'])
total_markers['delta'] = total_markers['cluster_from_rate'] - total_markers['cluster_to_rate']
total_markers = total_markers[(total_markers['delta'] < -0.3)].copy()

In [17]:
assert total_markers.set_index(['gene_id', 'cluster_from', 'cluster_to']).index.duplicated().sum() == 0

## Save results

In [18]:
total_markers.to_msgpack('TotalPairwiseMarker.msg')

In [19]:
marker_counts = total_markers.set_index(['cluster_from', 'cluster_to']).index.value_counts()
marker_counts.index = pd.MultiIndex.from_tuples(marker_counts.index.tolist())
marker_counts = marker_counts.reset_index()
marker_counts.columns = ['ClusterA', 'ClusterB', 'GeneCount']
marker_counts.to_csv('Cluster_pair_marker_counts.csv', index=None)

In [20]:
marker_counts[marker_counts['GeneCount'] < 10]

,ClusterA,ClusterB,GeneCount


In [21]:
subprocess.run('rm -rf TEMP*', shell=True)

CompletedProcess(args='rm -rf TEMP*', returncode=0)

In [22]:
with open('TotalGeneID.txt', 'w') as f:
    for g in total_markers['gene_id'].unique():
        f.write(f'{g}\n')

In [23]:
total_markers['gene_id'].unique().size

24438